# Create Data Tables For ONT Data QC<a class="tocSkip">

**This notebook automatically reads in data stored in the AnVIL_HPRC workspace's bucket and generates data table so the data can be run through QC (NTSM and Coverage's WDLs)**

**Jeltje changed this in Feb 2023 to output per-file tables instead of per-sample**

**Below are the steps taken in this notebook:**
1. Import Statements & Global Variable Definitions
2. Define Functions
3. Read In Sample Names
4. Create Dataframe Of Files
5. Write data frame to data tables

# Import Statements & Global Variable Definitions

## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install terra-pandas
#%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [1]:
from firecloud import fiss
import pandas as pd         
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations
Update this with project specific info

In [2]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

sample_info_fn = 'UCSC_HPRC_ONT_Y3_GUPPY6_samples.tsv' # a list of IDs,e.g. HG01243
sample_info_fn_na = 'UCSC_HPRC_ONT_Y3_GUPPY6_samples_GM_NA.tsv' # a list of IDs,e.g. NA01243
subm_key       = '79275EDA-C282-424A-9D5B-A8E876592893--UCSC_HPRC_ONT_Y3_GUPPY6'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_YEAR3_ONT_GUPPY6
Workspace storage bucket: gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/


# Function Definitions

In [3]:
def rtn_datatype_ls_for_subm(bucket_url, submission_key, data_type_subdir, file_type_ls):
    '''Takes in:
            * bucket_url (string): url of bucket to search (should be the AnVIL_HPRC bucket)
                ex: "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"
            * submission_key (string): the UUID plus the submission name to search
                ex: "5c68b972-8534-402f-9861-11c93558765f--UW_HPRC_HiFi_Y3"
            * data_type_subdir (string): name of the subfolder to search 
              (used when a submission has more than one data type.)
                ex: "PacBio_HiFi" if the data is not in subfolders, pass "."
            * file_type_ls (list of strings): file extensions to search for. Often a submission will 
              have more than one type of file that represents the same dataset. Be careful to not 
              include replicate data, however.
                ex: ".hifi_reads.bam"
     then performs a list of the bucket, then returns a filtered list files.'''
    
    rtn_file_ls = []
    
    submission_path = str(bucket_url + "submissions/" + submission_key)
    file_list_byt   = subprocess.run(['gsutil', '-u', 'human-pangenome-ucsc', 'ls', '-r', submission_path], 
                                     stdout=subprocess.PIPE)

    file_list_str   = file_list_byt.stdout.decode('utf-8')
    file_list       = file_list_str.split('\n')  ## Pull out "\n"
   
    ## filter out empty strings
    file_list = [ elem for elem in file_list if elem != '']
    
    ## Pull from subdir type we are targeting
    file_list = list(filter(lambda x:re.search(rf"{data_type_subdir}", x), file_list))
    
    for file_type in file_type_ls:
    
        ## Pull files of correct type (ex: ccs.bam)
        file_list_by_type = list(filter(lambda x:re.search(rf"{file_type}$", x), file_list))

        ## Add to list of files to return
        rtn_file_ls += file_list_by_type

    return rtn_file_ls    

# Read In Sample Names

In [4]:
## This file (a list of sample IDs) should be stored in the Terra workspace (in the bucket) that is being used for 
## the submission you  are wrangling...
sample_info_fp = os.path.join(bucket + sample_info_fn_na) 

sample_df = pd.read_csv(sample_info_fp, header=None, sep="\t")
sample_df.rename(columns = {0:'sample_id',1:'1000G'}, inplace = True)
sample_df.head()

,sample_id,1000G
0,GM18522,NA18522
1,GM18570,NA18570
2,GM18612,NA18612
3,GM18747,NA18747
4,GM18971,NA18971


# Create Dataframe Of Files

## Find the data

In [5]:
data_type_subdir = "."
file_type_ls     = ["pass.bam"]

## get a list of the files in submission (subm_key) that match the 
## data_type_subdir and file_type_ls
file_ls  = rtn_datatype_ls_for_subm(anvil_hprc_bucket, subm_key, 
                                      data_type_subdir, file_type_ls)

## Check that the number of files matches what we expect
len(file_ls)

217

## Add Sample Data To File Data Frame

In [6]:
filedict = dict()

for f in file_ls:
    for s in sample_df.sample_id:
        if re.search(s,f):
            filedict[f] = s
            break            
print(len(filedict))
file_df = pd.DataFrame.from_dict(filedict.items())
file_df.columns = ['ONT_pass_bam', 'sample']
file_df = file_df.merge(sample_df, left_on='sample', right_on='sample_id')
file_df = file_df[['ONT_pass_bam', 'sample', '1000G' ]]
file_df.head()


217


,ONT_pass_bam,sample,1000G
0,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522
1,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522
2,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522
3,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18570,NA18570
4,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18570,NA18570


## Add 1000G data (for NTSM run)

In [44]:
## Read in 1000G data from another workspace
## We will be using this to compare against out submission to make sure that
## the data comes from the same samples
#thousand_genomes_df = table_to_dataframe("sample", 
#                                        workspace="1000G-high-coverage-2019", 
#                                       workspace_namespace="anvil-datastorage")
#thousand_genomes_df.head()
## Use the 1kg library name (i.e. HG00621) as the index (that matches our sample name)
#thousand_genomes_df.set_index('library_name', inplace=True)

## We only need the cram file (represents 30X Ilmn dataset)
#thousand_genomes_df = thousand_genomes_df[['cram']]

## name the column to be a bit more descriptive
#thousand_genomes_df.rename(columns = {'cram':'1000g_cram'}, inplace = True)

## Sometimes some of the samples are not present in the 1000g file
#addSamples = list(set(file_df['sample']) - set(thousand_genomes_df.index.tolist()))
#print('missing:', addSamples)

In [7]:
## Read in 1000G data from another workspace
## instead of cram files (as before) we can use fastq file so we're not dependent
## on the public hg18.fa file, which has been giving problems.
## We will be using this to compare against out submission to make sure that
## the data comes from the same samples

t2t_reads_df = table_to_dataframe("participant", 
                                 workspace_namespace="anvil-datastorage", 
                                 workspace="AnVIL_T2T")
## Ensure that 1kg library id (i.e. HG00621) as the index (that matches our sample name)


## We only need the fastq files (represents 30X Ilmn dataset)
t2t_reads_df = t2t_reads_df[['read_1_fastq','read_2_fastq']]
t2t_reads_df["fastq_list"] = list(t2t_reads_df.values)

## name the column to be a bit more descriptive
#t2t_reads_df.rename(columns = {'read_1_fastq':'read_1_fastq_T2T'}, inplace = True)
#t2t_reads_df.rename(columns = {'read_2_fastq':'read_2_fastq_T2T'}, inplace = True)


## Sometimes some of the samples are not present in the 1000g file
addSamples = list(set(file_df['1000G']) - set(t2t_reads_df.index.tolist()))
print('missing:', addSamples)

t2t_reads_df

missing: []


,read_1_fastq,read_2_fastq,fastq_list
participant_id,,,
HG00096,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
HG00097,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
HG00099,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
HG00100,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
HG00101,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
...,...,...,...
NA21137,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
NA21141,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
NA21142,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...


## If there are any missing samples

In [8]:
## For the missing files we're going to get genome info from anvil_HPRC
## the data comes from the same samples
anvil_hprc_df = table_to_dataframe("sample", 
                                        workspace="AnVIL_HPRC", 
                                        workspace_namespace="anvil-datastorage")
## sample_id is the index
anvil_hprc_df = anvil_hprc_df[['child_ilmn', 'hic']]

## we want the child_ilnm value for HG002 instead of the hic value
anvil_hprc_df.loc[['HG002']]['hic'] = anvil_hprc_df.loc[['HG002']]['child_ilmn']

## now only retain the one column
anvil_hprc_df = anvil_hprc_df[['hic']]

## name the column to match 1000genomes
anvil_hprc_df.rename(columns = {'hic':'1000g_cram'}, inplace = True)

# and concatenate the two
thousand_genomes_df = pd.concat([anvil_hprc_df.loc[addSamples], thousand_genomes_df], ignore_index=False)

# check
list(set(file_df['sample']) - set(thousand_genomes_df.index.tolist()))

KeyError: "None of [Index(['GM18570', 'GM19159', 'GM20858', 'GM19468', 'GM18522', 'GM18612',\n       'GM18983', 'GM19087', 'GM20752', 'GM18971', 'GM20905', 'GM19120',\n       'GM19043', 'GM20805', 'GM20799', 'GM18747', 'GM19391', 'GM19338',\n       'GM19185'],\n      dtype='object', name='sample_id')] are in the [index]"

In [8]:
## merge the two dataframes
ntsm_df = pd.merge(
    file_df,
    t2t_reads_df,
    left_on='1000G',
    right_on='sample',
    right_index=True)
ntsm_df.head()

,ONT_pass_bam,sample,1000G,read_1_fastq,read_2_fastq,fastq_list
0,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
1,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
2,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18522,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
3,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18570,NA18570,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...
4,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,GM18570,NA18570,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...


In [9]:
print(ntsm_df.shape, file_df.shape)

(217, 6) (217, 3)


# Upload To Tables

In [10]:
## Create tables for running NTSM and ReadStats
dataframe_to_table("ntsm",      ntsm_df, WORKSPACE, PROJECT)
#dataframe_to_table("readstats", file_df, WORKSPACE, PROJECT)